In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# SmartTrip — Personal Travel Itinerary Agent
### Capstone Project (Concierge Agents Track)

**Problem:**  
Travel planning is time-consuming. People spend hours researching attractions, building schedules, and organizing trips.

**Solution:**  
SmartTrip is a multi-agent travel itinerary generator using:
- InfoGatherer Agent
- Planner Agent
- Scheduler Agent
- Memory System
- AI-based text refinement (LLM)

It creates personalized multi-day travel plans quickly and automatically.

**Features Demonstrated:**
- ✔ Multi-Agent System (3 agents working sequentially)
- ✔ Tool Usage (LLM + custom code tool)
- ✔ Memory System (JSON MemoryBank)
- ✔ Observability (logging)
- ✔ Agent Evaluation Support (trip structure)
- ✔ Modular Architecture

This notebook contains:
1. Agent definitions  
2. Memory system  
3. Orchestration  
4. Complete trip generation demo  



In [2]:
!pip install openai --quiet

import os
import json
import time
import logging

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger("SmartTrip")

print("Setup complete.")




Setup complete.


In [3]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
OPENAI_API_KEY = user_secrets.get_secret("OPENAI_API_KEY")

print("Loaded key:", "YES" if OPENAI_API_KEY else "NO")



Loaded key: YES


In [4]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

print("Client ready.")




Client ready.


In [5]:
# LLM helper function using the new OpenAI API format

def llm_complete(prompt: str, max_tokens: int = 300):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",   # works with your key
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=0.3
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"LLM Error: {e}")
        return "LLM error occurred."


In [6]:
# Simple JSON memory system

class MemoryBank:
    def __init__(self, path="memory_bank.json"):
        self.path = path
        self._load()

    def _load(self):
        if os.path.exists(self.path):
            with open(self.path, "r") as f:
                self.store = json.load(f)
        else:
            self.store = {"users": {}}

    def save(self):
        with open(self.path, "w") as f:
            json.dump(self.store, f, indent=2)

    def get_user(self, user_id):
        return self.store["users"].get(user_id, {"preferences": {}, "trips": []})

    def update_user(self, user_id, data):
        self.store["users"][user_id] = data
        self.save()

    def add_trip(self, user_id, trip):
        user = self.get_user(user_id)
        user["trips"].append(trip)
        self.update_user(user_id, user)

memory = MemoryBank()

print("Memory system ready.")


Memory system ready.


In [7]:
# Agent classes: InfoGatherer, Planner, Scheduler

class InfoGatherer:
    def __init__(self):
        self.name = "InfoGatherer"

    def run(self, destination, preferences):
        logger.info(f"{self.name}: Collecting information for {destination}")

        # Mocked POI data (works even without internet)
        pois = [
            {"name": "Eiffel Tower", "desc": "Famous landmark in Paris"},
            {"name": "Louvre Museum", "desc": "World’s largest art museum"},
            {"name": "Seine River Cruise", "desc": "Relaxing river tour"},
        ]

        # LLM refinement (optional)
        refine_prompt = f"Summarize attractions in {destination}: {pois}"
        summary = llm_complete(refine_prompt)

        return {"pois": pois, "summary": summary}


class Planner:
    def __init__(self):
        self.name = "Planner"

    def run(self, gathered_data, days, preferences):
        logger.info(f"{self.name}: Planning {days} days")

        pois = gathered_data["pois"]
        plan = []

        for d in range(days):
            poi = pois[d % len(pois)]
            plan.append({
                "day": d + 1,
                "activities": [
                    {"time": "09:00", "plan": f"Visit {poi['name']}"},
                    {"time": "13:00", "plan": "Lunch Break"},
                    {"time": "15:00", "plan": "Explore nearby areas"},
                ]
            })

        summary_prompt = f"Create a friendly summary for this plan: {plan}"
        summary = llm_complete(summary_prompt)

        return {"plan": plan, "summary": summary}


class Scheduler:
    def __init__(self):
        self.name = "Scheduler"

    def run(self, plan_data):
        logger.info(f"{self.name}: Adding basic travel time")

        for day in plan_data["plan"]:
            for act in day["activities"]:
                act["travel_hours"] = 0.2

        return plan_data


In [8]:
# Main SmartTrip Agent that coordinates all other agents

class SmartTripAgent:
    def __init__(self):
        self.info = InfoGatherer()
        self.planner = Planner()
        self.scheduler = Scheduler()

    def build_itinerary(self, user_id, destination, days=3, preferences=None):
        logger.info(f"SmartTripAgent: Building itinerary for {destination}")

        # Load or update preferences in memory
        user_data = memory.get_user(user_id)

        if preferences:
            user_data["preferences"].update(preferences)
            memory.update_user(user_id, user_data)

        # Step 1 — Gather info
        gathered = self.info.run(destination, user_data["preferences"])

        # Step 2 — Create plan
        plan = self.planner.run(gathered, days, user_data["preferences"])

        # Step 3 — Add travel timing
        scheduled = self.scheduler.run(plan)

        # Save trip to memory
        trip_record = {
            "destination": destination,
            "days": days,
            "preferences": user_data["preferences"],
            "itinerary": scheduled,
            "timestamp": time.time()
        }

        memory.add_trip(user_id, trip_record)

        return trip_record


In [9]:
# Demo / Test Run

agent = SmartTripAgent()

trip = agent.build_itinerary(
    user_id="demo_user",
    destination="Paris",
    days=3,
    preferences={"pace": "relaxed", "interests": ["art", "food"]}
)

print("=== SUMMARY ===")
print(trip["itinerary"]["summary"])

print("\n=== FULL ITINERARY ===")
for day in trip["itinerary"]["plan"]:
    print(f"\nDay {day['day']}:")
    for act in day["activities"]:
        print(f"  {act['time']} - {act['plan']} (travel ~{act['travel_hours']} hrs)")


2025-11-30 18:37:26,426 INFO SmartTripAgent: Building itinerary for Paris
2025-11-30 18:37:26,427 INFO InfoGatherer: Collecting information for Paris
2025-11-30 18:37:28,337 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-11-30 18:37:28,339 INFO Retrying request to /chat/completions in 0.435537 seconds
2025-11-30 18:37:28,906 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-11-30 18:37:28,908 INFO Retrying request to /chat/completions in 0.761145 seconds
2025-11-30 18:37:29,760 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-11-30 18:37:29,762 ERROR LLM Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota'

=== SUMMARY ===
LLM error occurred.

=== FULL ITINERARY ===

Day 1:
  09:00 - Visit Eiffel Tower (travel ~0.2 hrs)
  13:00 - Lunch Break (travel ~0.2 hrs)
  15:00 - Explore nearby areas (travel ~0.2 hrs)

Day 2:
  09:00 - Visit Louvre Museum (travel ~0.2 hrs)
  13:00 - Lunch Break (travel ~0.2 hrs)
  15:00 - Explore nearby areas (travel ~0.2 hrs)

Day 3:
  09:00 - Visit Seine River Cruise (travel ~0.2 hrs)
  13:00 - Lunch Break (travel ~0.2 hrs)
  15:00 - Explore nearby areas (travel ~0.2 hrs)


## SmartTrip — Personal Travel Itinerary Agent  
### Concierge Agents Track — Capstone Project

### 🔹 Problem
Planning a multi-day trip is time-consuming. People must search for attractions, decide which places to visit each day, and estimate timing.  
This requires time, research, and manual effort.

### 🔹 Solution
SmartTrip is an AI-driven travel itinerary agent that automatically builds structured multi-day itineraries.  
It combines multiple cooperating agents in a pipeline:

**1. InfoGatherer Agent**  
- Retrieves attractions (mocked or via LLM)  
- Summarizes highlights using the LLM

**2. Planner Agent**  
- Creates a day-by-day itinerary  
- Generates user-friendly descriptions

**3. Scheduler Agent**  
- Adds basic travel time  
- Polishes the timing structure

**4. Memory System**  
- Saves user preferences  
- Stores past trips  
- Enables persistent personalization

### 🔹 Key Concepts Demonstrated (Required by Kaggle)
- **Multi-Agent System:** InfoGatherer → Planner → Scheduler  
- **Tools:**  
  - LLM tool  
  - Custom calculations  
- **Memory & Session:** JSON-based memory bank  
- **Observability:** Logging throughout agent pipeline  
- **Evaluation Support:** Structured itinerary output  
- **Architecture:** Fully modular & extensible

### 🔹 Architecture Diagram (Text Version)
User → SmartTripAgent →  
&nbsp;&nbsp;→ InfoGatherer  
&nbsp;&nbsp;→ Planner  
&nbsp;&nbsp;→ Scheduler  
→ Final Itinerary  
→ MemoryBank

### 🔹 Value
- Reduces hours of manual planning  
- Produces clean multi-day itineraries  
- Easy to extend with real-time search tools  
- Useful for travelers, bloggers, agencies, students

### 🔹 How to Run
1. Add API key in Kaggle Secrets (optional for LLM)  
2. Run all cells top-to-bottom  
3. Use the demo cell to generate itineraries  

### 🔹 Future Improvements
- Replace mocked POIs with Google Search Tool  
- Add parallel agent for POI ranking  
- Add advanced scheduler  
